In [1]:
import pandas as pd
import re
import numpy as np
import sys
#from textdistance import levenshtein
#import  pyxdameraulevenshtein as dl
#from fuzzywuzzy import process, fuzz
sys.path.append('C:/Users/USER/Box/Data School choice/Data Merge')
#sys.path.append('/Users/laura/Documents/ThirdYear/School Choice project/Data Merge')
from dictionarySchools import mydict
import warnings
warnings.filterwarnings("ignore")

## csec

In [2]:
def read_excel(excel_file):
    return pd.read_excel(excel_file)

In [3]:
def select_columns(df):
    
    df.columns = df.columns.str.upper()
    df.columns = ['FIRST_NAME' if 'FIRST' in x else x for x in df.columns]
    df.columns = ['LAST_NAME' if 'LAST' in x else x for x in df.columns]
    df.columns = ['DATE_OF_BIRTH' if 'DOB' in x else x for x in df.columns]
    df.columns = ['OVERALL_GRADE' if 'OVERAL' in x else x for x in df.columns]
    df.columns = ['SUBJECT' if 'SUBJ' in x else x for x in df.columns]
    df.columns = ['SCHOOL' if 'SCHOOL' in x else x for x in df.columns]
    
    df = df[['FIRST_NAME', 'LAST_NAME', 'DATE_OF_BIRTH', 'SEX', 'SCHOOL', 'SUBJECT', 'OVERALL_GRADE']]
    
    return df

In [4]:
def handle_school(df):
    
    df["SCHOOL"] = df["SCHOOL"].str.replace("-","")
    df["SCHOOL"] = df["SCHOOL"].str.replace("'","")
    df["SCHOOL"] = df["SCHOOL"].str.replace(" ","")
    df["SCHOOL"] = df["SCHOOL"].str.replace(".","")
    df["SCHOOL"].replace(mydict, inplace=True)
    
    return df

In [5]:
def concat_csecs(file1, file2, file3, file4, file5, file6):
    
    csec1 = read_excel(file1)
    csec2 = read_excel(file2)
    csec3 = read_excel(file3)
    csec4 = read_excel(file4)
    csec5 = read_excel(file5)
    csec6 = read_excel(file6)
    
    df1 = select_columns(csec1)
    df2 = select_columns(csec2)
    df3 = select_columns(csec3)
    df4 = select_columns(csec4)
    df5 = select_columns(csec5)
    df6 = select_columns(csec6)
    
    df1 = handle_school(df1)
    df2 = handle_school(df2)
    df3 = handle_school(df3)
    df4 = handle_school(df4)
    df5 = handle_school(df5)
    df6 = handle_school(df6)

    pre_name = 'PRE_' + str(''.join(filter(str.isdigit, str(file5))))
    df1.loc[:, pre_name] = 1
    df2.loc[:, pre_name] = 1
    df3.loc[:, pre_name] = 1
    df4.loc[:, pre_name] = 1
    df5.loc[:, pre_name] = 0
    df6.loc[:, pre_name] = 0
    
    concat_df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index = True)
    
    return concat_df

In [6]:
def handle_grades(df, pre_name):
    
    df['OVERALL_GRADE'].fillna(0, inplace=True)
    
    df.OVERALL_GRADE.replace({'VI': 1, 'V': 2, 'IV': 3, 'III': 4, 'II': 5, 'I':6,
                              'ABS':0, 'UNG+':0, 'UNG*':0, 'CANC':0, 'UNG-':0, 'UNG':0, 'WIT':0,
                              'CAN':0, 'WITH':0, 'WITHHELD':0, 'CANCELLED':0, 'L':0}, inplace=True)
    
    df  = df.drop_duplicates(subset = ['FIRST_NAME','MIDDLE_NAME','LAST_NAME','DATE_OF_BIRTH',
                                       'SEX','SCHOOL','SUBJECT','OVERALL_GRADE',
                                       pre_name, 'IDENTIFIER1','IDENTIFIER2','IDENTIFIER3'], keep='first')
    
    return df
    

In [7]:
def agg_school(df):
    
    df_schools_temp = df.drop_duplicates(subset = ['FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME',
                                          'DATE_OF_BIRTH', 'SEX','SCHOOL','IDENTIFIER1',
                                          'IDENTIFIER2','IDENTIFIER3'],keep='first')
    
    df_schools = df_schools_temp.groupby(['FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME',
                                         'DATE_OF_BIRTH', 'SEX','IDENTIFIER1','IDENTIFIER2',
                                          'IDENTIFIER3']).agg({'SCHOOL': '%'.join}).reset_index()
    
    df_schools['SCHOOL'] = df_schools['SCHOOL'].apply(lambda x :np.where('%' in x, 'multiple%' + x, x))
    
    return df_schools

In [8]:
def agg_pre(df, pre_name):
    
    df_PRE = df.groupby(['FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME',
                        'DATE_OF_BIRTH', 'SEX', 'IDENTIFIER1','IDENTIFIER2', 
                        'IDENTIFIER3']).agg({pre_name: 'max'}).reset_index()
    
    return df_PRE

In [9]:
def manipulate_df(df):
    
    df[['FIRST_NAME', 'MIDDLE_NAME']] = df["FIRST_NAME"].str.rsplit(expand=True, n=1) 
    df['MIDDLE_NAME'] = df['MIDDLE_NAME'].fillna('')
    df['FIRST_NAME'] = df['FIRST_NAME'].map(lambda x: re.sub(r'[^a-zA-Z]', '', x))
    df['MIDDLE_NAME'] = df['MIDDLE_NAME'].map(lambda x: re.sub(r'[^a-zA-Z]', '', str(x)))
    df['LAST_NAME'] = df['LAST_NAME'].map(lambda x: re.sub(r'[^a-zA-Z]', '', x))
    
    df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], errors='coerce')
    df['DATE_OF_BIRTH'] = df["DATE_OF_BIRTH"].dt.strftime("%d%b%y")
    
    df['IDENTIFIER1'] = df['FIRST_NAME'].astype(str) + df['LAST_NAME'].astype(str) + df['DATE_OF_BIRTH'].astype(str) + df['SEX'].astype(str)
    df['IDENTIFIER2'] = df['FIRST_NAME'].astype(str) + df['MIDDLE_NAME'].astype(str) + df['LAST_NAME'].astype(str) + df['DATE_OF_BIRTH'].astype(str) + df['SEX'].astype(str)
    df['IDENTIFIER3'] = df['FIRST_NAME'].astype(str) + df['MIDDLE_NAME'].apply(lambda x: '' if len(x)==0 else x[0])+ df['LAST_NAME'].astype(str) + df['DATE_OF_BIRTH'].astype(str)+ df['SEX'].astype(str)
    
    return df

In [10]:
def pivot_df(df):
    
    df.OVERALL_GRADE = df.OVERALL_GRADE.astype(int)
    
    new_table1 = df.pivot_table(values='OVERALL_GRADE', 
                                index=['FIRST_NAME','MIDDLE_NAME','LAST_NAME', 
                                       'SEX', 'DATE_OF_BIRTH',
                                       'IDENTIFIER1','IDENTIFIER2','IDENTIFIER3'], 
                                columns='SUBJECT').reset_index()
    
    return new_table1

In [11]:
def csec_process(file1, file2, file3, file4, file5, file6):
    df = concat_csecs(file1, file2, file3, file4, file5, file6)
    pre_name = 'PRE_' + str(''.join(filter(str.isdigit, str(file5))))
    
    df = manipulate_df(df)
    df = handle_grades(df, pre_name)   
    df_schools = agg_school(df)
    df_PRE = agg_pre(df, pre_name)
    
    new_table1 = pivot_df(df)
    new_table2 = pd.merge(new_table1, df_schools, how="inner", on = ['FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME',
                                                                     'DATE_OF_BIRTH','SEX',
                                                                     'IDENTIFIER1','IDENTIFIER2','IDENTIFIER3'])
    new_table = pd.merge(new_table2, df_PRE, how="inner", on = ['FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME',
                                                                 'DATE_OF_BIRTH','SEX',
                                                                'IDENTIFIER1','IDENTIFIER2','IDENTIFIER3'])
    new_table['YEAR'] = pd.DatetimeIndex(new_table['DATE_OF_BIRTH']).year
    new_table['indexCSEC'] = new_table.index
    
    
    start_year = str(''.join(filter(str.isdigit, str(file1))))
    end_year = str(''.join(filter(str.isdigit, str(file6))))
    output_name = 'CSEC_CON_' + str(start_year) + '_' + str(end_year) + '.xlsx'
    new_table.to_excel(output_name, encoding='utf-8', index=False)
    
    return new_table


## csec inputs here
Note: Years should be in ascending order

In [12]:
df_csec = csec_process('tb2008.xlsx', 'tb2009.xlsx', 'tb2010.xlsx', 'tb2011.xlsx', 'tb2012.xlsx', 'tb2013.xlsx')
df_csec

,FIRST_NAME,MIDDLE_NAME,LAST_NAME,SEX,DATE_OF_BIRTH,IDENTIFIER1,IDENTIFIER2,IDENTIFIER3,ADDITIONAL MATHEMATICS,AGRICULTURAL SCIENCE DOU,...,SOCIAL STUDIES,SPANISH,TECHNICAL DRAWING,THEATRE ARTS,TYPEWRITING,VISUAL ARTS,SCHOOL,PRE_2012,YEAR,indexCSEC
0,AADEEN,PETAGAYE,SMITH,F,18Jun92,AADEENSMITH18Jun92F,AADEENPETAGAYESMITH18Jun92F,AADEENPSMITH18Jun92F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,multiple%DENBIGHHIGHSCHOOL%MATHUNLIMITED,1,1992,0
1,AADRENE,,BROWN,F,23Jul90,AADRENEBROWN23Jul90F,AADRENEBROWN23Jul90F,AADRENEBROWN23Jul90F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BROWNSTOWNHIGHSCHOOL(EVENING),1,1990,1
2,AADRIEN,SEAN,MCDERMOTT,M,29Oct90,AADRIENMCDERMOTT29Oct90M,AADRIENSEANMCDERMOTT29Oct90M,AADRIENSMCDERMOTT29Oct90M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,EXCELSIORHIGHSCHOOL,1,1990,2
3,AADRONIE,,INGRAM,M,02May92,AADRONIEINGRAM02May92M,AADRONIEINGRAM02May92M,AADRONIEINGRAM02May92M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BROWNSTOWNHIGHSCHOOL,1,1992,3
4,AAGEL,A,MARSHALL,F,10Mar96,AAGELMARSHALL10Mar96F,AAGELAMARSHALL10Mar96F,AAGELAMARSHALL10Mar96F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MEADOWBROOKHIGHSCHOOL,0,1996,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388471,ZWARDE,K,LOGAN,F,27Jul87,ZWARDELOGAN27Jul87F,ZWARDEKLOGAN27Jul87F,ZWARDEKLOGAN27Jul87F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SCHOOLOFEXCELLENCE,1,1987,388471
388472,ZWELI,,REID,M,20Apr95,ZWELIREID20Apr95M,ZWELIREID20Apr95M,ZWELIREID20Apr95M,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,multiple%HERBERTMORRISONTECHNICALSCHOOL%HERBER...,0,1995,388472
388473,ZWELI,KHALIL,REID,M,20Apr95,ZWELIREID20Apr95M,ZWELIKHALILREID20Apr95M,ZWELIKREID20Apr95M,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,PRIVATE SCHOOL,0,1995,388473
388474,ZYANE,M,BLACKWOOD,M,01May90,ZYANEBLACKWOOD01May90M,ZYANEMBLACKWOOD01May90M,ZYANEMBLACKWOOD01May90M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SCHOOLOFEXCELLENCE:SANTACRUZ,0,1990,388474


## GSAT

In [24]:
def manipulate_gsat(df_gsat):
    
    df_gsat['MNAME.1'] = df_gsat['MNAME.1'].fillna('')
    df_gsat['DOB_CLEAN'] = pd.to_datetime(df_gsat['DOB Converted'], errors='coerce')
    df_gsat['DOB_CLEAN'] = df_gsat["DOB_CLEAN"].dt.strftime("%d%b%y")
    df_gsat['FNAME.1'] = df_gsat['FNAME.1'].map(lambda x: re.sub(r'[^a-zA-Z]', '', str(x)))
    df_gsat['MNAME.1'] = df_gsat['MNAME.1'].map(lambda x: re.sub(r'[^a-zA-Z]', '', str(x)))
    df_gsat['SURNAME.1'] = df_gsat['SURNAME.1'].map(lambda x: re.sub(r'[^a-zA-Z]', '', str(x)))
    df_gsat['IDENTIFIER1'] = df_gsat['FNAME.1'].astype(str) + df_gsat['SURNAME.1'].astype(str) + df_gsat['DOB_CLEAN'].astype(str) + df_gsat['GENDER'].astype(str)
    df_gsat['IDENTIFIER2'] = df_gsat['FNAME.1'].astype(str) + df_gsat['MNAME.1'].astype(str) + df_gsat['SURNAME.1'].astype(str) + df_gsat['DOB_CLEAN'].astype(str) + df_gsat['GENDER'].astype(str)
    df_gsat['IDENTIFIER3'] = df_gsat['FNAME.1'].astype(str) + df_gsat['MNAME.1'].apply(lambda x: '' if len(x)==0 else x[0])+ df_gsat['SURNAME.1'].astype(str) + df_gsat['DOB_CLEAN'].astype(str)+ df_gsat['GENDER'].astype(str)
    df_gsat['PLACEDN'] = df_gsat['PLACEDN'].map(lambda x: re.sub(r'[\s.-]', '', str(x)))
    df_gsat['PLACEDN'] = df_gsat['PLACEDN'].str.replace("'","")
    df_gsat["PLACEDN"].replace(mydict, inplace=True)
    
    return df_gsat

In [14]:
def handle_duplicates(df, ID):
    return df.drop_duplicates(subset = [ID], keep = False)

In [15]:
def merge_step1(large_gsat_df, gsat_df, csec_df):
    
    df_inner_id2 = pd.merge(gsat_df, csec_df, on ='IDENTIFIER2', how ='inner')
    df_duplicates_s1 = large_gsat_df[~large_gsat_df.IDENTIFIER2.isin(gsat_df.IDENTIFIER2)]
    df_gsat_id2_nm_s1 = gsat_df[~gsat_df.IDENTIFIER2.isin(df_inner_id2.IDENTIFIER2)]
    
    return df_inner_id2, df_duplicates_s1, df_gsat_id2_nm_s1

In [16]:
def merge_step2(gsat_df, csec_df):
    
    df_gsat_id2_nm_s1_nd = gsat_df.drop_duplicates(subset = ['IDENTIFIER3'], keep = False)
    df_duplicates_s2 = gsat_df[~gsat_df.IDENTIFIER3.isin(df_gsat_id2_nm_s1_nd.IDENTIFIER3)]
    df_inner_id3 = pd.merge(df_gsat_id2_nm_s1_nd, csec_df, on='IDENTIFIER3', how = 'inner')
    df_gsat_id2_nm_s2 = df_gsat_id2_nm_s1_nd[~df_gsat_id2_nm_s1_nd.IDENTIFIER3.isin(df_inner_id3.IDENTIFIER3)]
    
    return df_inner_id3, df_duplicates_s2, df_gsat_id2_nm_s2

In [17]:
def handle_unmatched_df(df_duplicates_s1, df_duplicates_s2, df):
    
    unmatchedRound = pd.concat([df_duplicates_s1, df_duplicates_s2, df])
    unmatchedRound['SCHOOL'] = unmatchedRound['PLACEDN']
    
    return unmatchedRound

In [18]:
def merge_step3(unmatched_df, csec_df):
    
    df_inner_id2_nm_school = pd.merge(unmatched_df, csec_df, on=['IDENTIFIER2', 'SCHOOL'], how = 'inner')
    df_gsat_id2_nm_school_nm_s3 = unmatched_df[~unmatched_df.CANDID.isin(df_inner_id2_nm_school.CANDID)]
    df_gsat_id2_nm_school_nm_s3 = df_gsat_id2_nm_school_nm_s3.drop_duplicates(subset = ['IDENTIFIER3', 'SCHOOL'], keep =False)
    
    return df_inner_id2_nm_school, df_gsat_id2_nm_school_nm_s3

In [19]:
def merge_step4(gsat_df, csec_df):
    
    df_inner_id3_nm_school = pd.merge(gsat_df, csec_df, on=['IDENTIFIER3','SCHOOL'], how = 'inner')
    df_gsat_id3_nm_school_nm_s4 = gsat_df[~gsat_df.CANDID.isin(df_inner_id3_nm_school.CANDID)]
    df_gsat_id3_nm_school_nm_s4 = df_gsat_id3_nm_school_nm_s4.drop_duplicates(subset = ['IDENTIFIER1','SCHOOL'], keep =False)
    
    return df_inner_id3_nm_school, df_gsat_id3_nm_school_nm_s4

In [20]:
def merge_step5(gsat_df, csec_df):
    
    df_inner_id1_nm_school = pd.merge(gsat_df, csec_df, on=['IDENTIFIER1', 'SCHOOL'], how = 'inner')
    df_gsat_id1_nm_school_nm_s5 = gsat_df[~gsat_df.CANDID.isin(df_inner_id1_nm_school.CANDID)]
    
    return df_inner_id1_nm_school, df_gsat_id1_nm_school_nm_s5

In [32]:
def handle_matched_all(df_inner_id2, df_inner_id3, matched_s3, matched_s4, matched_s5, pre_name):
    
    matched_All = pd.concat([df_inner_id2, df_inner_id3, matched_s3, matched_s4, matched_s5], ignore_index=True)
    matched_All.drop(['IDENTIFIER1_x', 'IDENTIFIER1_y','IDENTIFIER2_x', 'IDENTIFIER2_y','IDENTIFIER3_x', 'IDENTIFIER3_y'], axis=1, inplace=True)
    
    return matched_All

In [22]:
def match_gast_and_csec(gsat_file, df_csec):
    gsat = read_excel(gsat_file)
    pre_name = ''.join([x for x in list(df_csec.columns) if 'PRE_' in x ])
    
    df_gsat = manipulate_gsat(gsat)
    
    df_gsat_id1 = handle_duplicates(df_gsat, 'IDENTIFIER1') 
    df_gsat_id2 = handle_duplicates(df_gsat, 'IDENTIFIER2') 
    df_gsat_id3 = handle_duplicates(df_gsat, 'IDENTIFIER3') 
    new_table_id1 = handle_duplicates(df_csec, 'IDENTIFIER1') 
    new_table_id2 = handle_duplicates(df_csec, 'IDENTIFIER2')
    new_table_id3 = handle_duplicates(df_csec, 'IDENTIFIER3')
    
    df_inner_id2, df_duplicates_s1, df_gsat_id2_nm_s1 = merge_step1(df_gsat, df_gsat_id2, new_table_id2)
    df_inner_id3, df_duplicates_s2, df_gsat_id2_nm_s2 = merge_step2(df_gsat_id2_nm_s1, new_table_id3)
    
    unmatchedRound1 = handle_unmatched_df(df_duplicates_s1, df_duplicates_s2, df_gsat_id2_nm_s2)
    
    new_table_id1school = df_csec.drop_duplicates(subset = ['IDENTIFIER1', 'SCHOOL'], keep =False)
    new_table_id1school = new_table_id1school[~new_table_id1school['SCHOOL'].str.contains('multiple')]
    
    new_table_id2school = df_csec.drop_duplicates(subset = ['IDENTIFIER2', 'SCHOOL'], keep =False)
    new_table_id2school = new_table_id2school[~new_table_id2school['SCHOOL'].str.contains('multiple')]
    
    new_table_id3school = df_csec.drop_duplicates(subset = ['IDENTIFIER3', 'SCHOOL'], keep =False)
    new_table_id3school =  new_table_id3school[~new_table_id3school['SCHOOL'].str.contains('multiple')]
    
    matched_s3, df_gsat_id2_nm_school_nm_s3 = merge_step3(unmatchedRound1, new_table_id2school)
    matched_s4, df_gsat_id3_nm_school_nm_s4 = merge_step4(df_gsat_id2_nm_school_nm_s3, new_table_id3school)
    matched_s5, df_gsat_id1_nm_school_nm_s5 = merge_step5(df_gsat_id3_nm_school_nm_s4, new_table_id1school)
    
    matched_All = handle_matched_all(df_inner_id2, df_inner_id3, matched_s3, matched_s4, matched_s5, pre_name)
    
    output_name = 'matched_All_' + str(gsat_file)
    matched_All.to_excel(output_name, encoding='utf-8',index=False)
    
    
    return matched_All
    

## gsat input here

In [33]:
matched_df = match_gast_and_csec("GSAT_2007.xlsx", df_csec)
matched_df

,EXAMID,PERIODID,CANDID,FNAME,SURNAME,MNAME,SCHOOLID,SCHOOLN,GENDER,DOB,...,TECHNICAL DRAWING,THEATRE ARTS,TYPEWRITING,VISUAL ARTS,SCHOOL,PRE_2012,YEAR,indexCSEC,IDENTIFIER3,IDENTIFIER1
0,GSAT,2007.0,51985013.0,KRISTI,CHARLES,E,02427,SAINTS PETER AND PAUL PREPARATORY,F,13/02/1996,...,NaN,NaN,NaN,NaN,CAMPIONCOLLEGE,0,1996,178778,NaN,NaN
1,GSAT,2007.0,74234837.0,ROMARIO,WHITE,M,02256,DUHANEY PARK PRIMARY,M,22/01/1996,...,NaN,NaN,NaN,NaN,CAMPIONCOLLEGE,0,1996,276052,NaN,NaN
2,GSAT,2007.0,38466328.0,RACHEL,CHIN,L,02439,THE QUEEN'S PREPARATORY,F,11/02/1996,...,NaN,NaN,NaN,5.0,CAMPIONCOLLEGE,0,1996,255883,NaN,NaN
3,GSAT,2007.0,65836119.0,ASHLEIGH,YEATES,S,08406,MOUNT ALVERNIA PREPARATORY,F,03/02/1995,...,NaN,NaN,NaN,NaN,MOUNTALVERNIAHIGHSCHOOL,0,1995,31250,NaN,NaN
4,GSAT,2007.0,94102931.0,ROMARIO,MAXWELL,A,02049,ST. RICHARD'S PRIMARY,M,10/03/1995,...,NaN,NaN,NaN,NaN,CAMPIONCOLLEGE,0,1995,275600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26300,GSAT,2007.0,59335166.0,SHANEKE,PARRIS,R,12038,NEW FOREST PRI. & JNR. HIGH & INFANT,F,01/05/1995,...,NaN,NaN,NaN,NaN,CROSSKEYSHIGHSCHOOL,0,1995,307871,NaN,SHANEKEPARRIS01May95F
26301,GSAT,2007.0,36956262.0,OSHANE,STEWART,O,12005,BETHABARA PRIMARY & JUNIOR HIGH,M,03/03/1995,...,NaN,NaN,NaN,NaN,CROSSKEYSHIGHSCHOOL,0,1995,244944,NaN,OSHANESTEWART03Mar95M
26302,GSAT,2007.0,14728164.0,HOWARD,NELSON,R,05028,HILLSIDE PRIMARY,M,06/11/1994,...,NaN,NaN,NaN,NaN,ISLINGTONHIGHSCHOOL,0,1994,113119,NaN,HOWARDNELSON06Nov94M
26303,GSAT,2007.0,29306179.0,CARLTON,LOGAN,J,12057,WOODLANDS PRIMARY,M,22/02/1995,...,0.0,NaN,NaN,NaN,CROSSKEYSHIGHSCHOOL,0,1995,43652,NaN,CARLTONLOGAN22Feb95M
